In [1]:
import geopandas as gpd
import pyarrow
import pandas as pd
import os

In [3]:
target_2631 = pd.read_parquet('data/GPS.nosync/filter/Sonoma_2631_new', engine='pyarrow')

In [4]:

def write_file(filename, info):
  text_file = open(filename, "w")
  text_file.write(info)
  text_file.close()

In [ ]:
start_index = 0
start_id = target_2631.loc[start_index]['caid']
#print("start index: %d, caid: %s" %(0, start_id))
end = 200000
count = 0
for index in range(0, end):
  data_id = target_2631.loc[index]['caid']
  if data_id != start_id:
    #print("end index: %d, caid: %s" %(index - 1, target_2631.loc[index - 1]['caid']))
    #print("total points: %d" %(index - start_index))
    print("%d start: %d, end: %d, total: %d, caid: %s" %(count, start_index, index - 1, (index - start_index), data_id))
    count+=1
    start_index = index
    start_id = data_id
    
    #print("start index: %d, caid: %s" %(index, data_id))
#print("end index: %d, caid: %s" %(end, target_2631.loc[end]['caid']))



In [21]:

res = 'LINESTRING('
df = pd.DataFrame(
    {'Latitude': [float(target_2631.loc[72]['latitude'])],
     'Longitude': [float(target_2631.loc[72]['longitude'])]})

#for index in range_:
start_index = 72
end_index = 266
is_first = True
for index in range(start_index,end_index):
  
  single_data = target_2631.loc[index]
  if ((single_data['latitude'] < 38.85) & (single_data['latitude'] > 38.099) & (single_data['longitude'] < -122.35) & (single_data['longitude'] > -123.55)):
    lat = float(single_data['latitude'])
    lon = float(single_data['longitude'])
    df.loc[len(df.index)] = [float(lat),float(lon)]
    if is_first:
      is_first = False
      res+= (str(lon) + ' ' + str(lat))
    else:
      res+= (',' + str(lon) + ' ' + str(lat))
res+=')'
df = df[0:-1]
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))
write_file('data/GPS.nosync/output/' + target_2631.loc[start_index]['caid']+'-points.txt', res)

In [31]:
start_index = 0
start_id = target_2631.loc[start_index]['caid']
print("start index: %d, caid: %s" %(0, start_id))
end = 200000
count = 0
is_first = True
res = 'LINESTRING('
for index in range(0, end):
  data_id = target_2631.loc[index]['caid']
  single_data = target_2631.loc[index]
  is_data_valid = (single_data['latitude'] < 38.85) & (single_data['latitude'] > 38.099) & (single_data['longitude'] < -122.35) & (single_data['longitude'] > -123.55)
  if data_id != start_id:
    # print("end index: %d, caid: %s" %(index - 1, target_2631.loc[index - 1]['caid']))
    # print("total points: %d" %(index - start_index))
    res+=')'
    if (index - start_index) > 49:
      write_file('data/GPS.nosync/points/' + str(count) +'-points.txt', res)
    is_first = True
    res = 'LINESTRING('
    count+=1
    start_index = index
    start_id = data_id
  lat = float(single_data['latitude'])
  lon = float(single_data['longitude'])
  #df.loc[len(df.index)] = [float(lat),float(lon)]
  if is_data_valid:
    if is_first:
      is_first = False
      res+= (str(lon) + ' ' + str(lat))
    else:
      res+= (',' + str(lon) + ' ' + str(lat))

    # print("start index: %d, caid: %s" %(index, data_id))
#print("end index: %d, caid: %s" %(end, target_2631.loc[end]['caid']))

start index: 0, caid: 0000dec3ca31d5b1b07771650fb6ee077059a5a579f0e35c30d5d68a0008feb6


In [2]:
streets = gpd.read_file('data/GPS.nosync/Streets/Streets.shp')

In [26]:
file_path1 = "data/GPS.nosync/output/sub-result-4/4-m_wkt.txt"
file_path2 = "data/GPS.nosync/output/sub-result-4/4-e_wkt.txt"
#check if file is present
if os.path.isfile(file_path1):
    #open text file in read mode
    text_file = open(file_path1, "r")
 
    #read whole file to a string
    wkt1 = text_file.read()
 
    #close file
    text_file.close()
if os.path.isfile(file_path2):
    #open text file in read mode
    text_file = open(file_path2, "r")
 
    #read whole file to a string
    wkt2 = text_file.read()
 
    #close file
    text_file.close()
d1 = {'col1': ['name1', 'name2'], 'wkt': wkt1}
d2 = {'col1': ['name1', 'name2'], 'wkt': wkt2}
df = pd.DataFrame(d1)
gs = gpd.GeoSeries.from_wkt(df['wkt'])
gdf1 = gpd.GeoDataFrame(df, geometry=gs, crs="EPSG:4326")
df = pd.DataFrame(d2)
gs = gpd.GeoSeries.from_wkt(df['wkt'])
gdf2 = gpd.GeoDataFrame(df, geometry=gs, crs="EPSG:4326")


In [8]:
def get_gdf_from_res(file_path):
  if os.path.isfile(file_path):
    #open text file in read mode
    text_file = open(file_path, "r")

    #read whole file to a string
    wkt = text_file.read()
    #close file
    text_file.close()
    if ',' in wkt:
      d = {'col1': ['name1', 'name2'], 'wkt': wkt}
      df = pd.DataFrame(d)
      gs = gpd.GeoSeries.from_wkt(df['wkt'])
      gdf = gpd.GeoDataFrame(df, geometry=gs, crs="EPSG:4326")
      gdf['points'] = gdf.apply(lambda x: [y for y in x['geometry'].coords], axis=1)
      return gdf
    else:
      print('file_path %s contains so few points!' % file_path)
      return gpd.GeoDataFrame({'A' : []})
  else:
    return gpd.GeoDataFrame({'A' : []})


In [ ]:
#from folium import plugins
#map = folium.Map(location=[38.447135, -122.76633], tiles='OpenStreetMap',zoom_start=12)
heat_data = [[0,0]]

for index in range(0,999):
  file_path = 'data/GPS.nosync/output/sub-result-' + str(index) + '/' + str(index) + '-m_wkt.txt'
  gdf = get_gdf_from_res(file_path)
  if not gdf.empty:
    print('get the data of index:%d' % index)
    for points in gdf['points']:
      for p in points:
        heat_data.append([p[1],p[0]])
print(len(heat_data))
heat_data = heat_data[1:]
print(len(heat_data))


In [ ]:
heat_data

In [ ]:
import folium
from folium import plugins

map = folium.Map(location=[38.447135, -122.76633], tiles='OpenStreetMap',zoom_start=12,max_zoom = 14, min_zoom=10)

plugins.HeatMap(heat_data,min_opacity=0.4,
                   radius=21, blur=12, 
                   max_zoom=4, min_zoom=8).add_to(map)
#plugins.HeatMap(heat_data).add_to(map)
map

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(64, 64))
streets.plot(ax=ax, color='black', LineWidth = 2)
#gdf2.plot(ax=ax, color='green',LineWidth = 2)
#gdf.plot(ax=ax, color='blue', LineWidth = 1)
#gdf1.plot(ax=ax, color='red',LineWidth = 1)

In [ ]:
# parts = streets[1:10000]
# parts['points'] = parts.apply(lambda x: [y for y in x['geometry'].coords], axis=1)
# len(parts)
gdf2['points'] = gdf1.apply(lambda x: [y for y in x['geometry'].coords], axis=1)
gdf1.head
gdf1

In [28]:
import folium
map = folium.Map(location = [4,10], tiles = "Stamen Terrain", zoom_start = 3)

In [ ]:
from folium import plugins

map = folium.Map(location=[38.447135, -122.76633], tiles='OpenStreetMap',zoom_start=12)

#heat_data = [gdf2.points[0],gdf2.points[1]]
#heat_data[1] = 
heat_data = [[point[1], point[0]] for point in gdf2['points'][0] ]
# for point in gdf2['points'][1]:
#   heat_data.append([point[1],point[0]])
for points in gdf2['points']:
  for p in points:
    heat_data.append([p[1],p[0]])
heat_data
plugins.HeatMap(heat_data,min_opacity=0.5,
                   radius=4, blur=4, 
                   max_zoom=1).add_to(map)
map


In [ ]:
heat_data

In [48]:
map.save("index.html")

In [ ]:
plugins.HeatMap(heat_data).add_to(map)
map